# Benchmarking the selectivity estimation for the overlap predicate
---
The following notebook contains a demo of the benchmark procedure used to evaluate the different selectivity estimations using the overlap predicate as an example. 

In [19]:
from range import Range
from main import add_to_dataframe
import numpy as np
import plotly.express as px
import pandas as pd

NB_RANGES = 10000
NB_TESTS = 100
MAX_RANGE_VAL = 10000
NB_BUCKETS = 100

# Generate some random ranges to simulate a database range content
rand_ranges = [Range(MAX_RANGE_VAL) for _ in range(NB_RANGES)]
rand_ranges = np.array(rand_ranges)

# Constant ranges to be compared with
# ex : serange_range && const_range[i]
const_ranges = [Range(MAX_RANGE_VAL) for _ in range(NB_TESTS)]
const_ranges = np.array(const_ranges)
const_ranges_len = [r.len() for r in const_ranges]

for i in range(1, 5):
    print(rand_ranges[-i])

[3582, 7411]
[5202, 5893]
[2887, 4604]
[2474, 4160]


### Verification functions

In [20]:
def range_overlaps(r1, r2):
    return r1.start < r2.start < r1.end or \
           r1.start < r2.end < r1.end or \
           r2.start < r1.start < r2.end or \
           r2.start < r1.end < r2.end

def real_overlap(rand_ranges: np.ndarray, const_ranges):
    nb_line = np.zeros(len(const_ranges))
    for idx, cr in enumerate(const_ranges):
        n = 0
        for rr in rand_ranges:
            if range_overlaps(cr, rr):
                n += 1
        nb_line[idx] = n
    return nb_line


### Building histograms

In [21]:
def build_belonging_hist(rand_ranges, bins):
    histo = np.zeros(NB_BUCKETS)

    n = 0
    for r in rand_ranges:
        # Compute the number of bins which this range overlaps
        end_idx, start_idx = bound_idx(bins, r)
        n += end_idx - start_idx + 1  # Nb of buckets this range overlaps
        for i in range(start_idx, end_idx + 1):
            if i >= len(histo):
                break
            histo[i] += 1
    n /= NB_RANGES
    return histo, n

def build_local_norm(rand_ranges, bins):
    histo = np.zeros(NB_BUCKETS)
    for r in rand_ranges:
        # Compute the number of bins which this range overlaps
        end_idx, start_idx = bound_idx(bins, r)
        n = end_idx - start_idx + 1  # Nb of buckets this range overlaps
        for i in range(start_idx, end_idx + 1):
            if i >= len(histo):
                break
            histo[i] += 1 / n
    return histo

def build_bound_hists(rand_ranges, bins):
    hist_up = np.zeros(NB_BUCKETS)
    hist_low = np.zeros(NB_BUCKETS)
    for r in rand_ranges:
        end_idx, start_idx = bound_idx(bins, r)
        hist_up[end_idx] += 1
        hist_low[start_idx] += 1
    return hist_low, hist_up

def build_histos(rand_ranges):
    """
    Build diverse histograms for the column containing rand_ranges
    :param rand_ranges: Column of range types
    :return: Many histos
    """
    max_val = rand_ranges.max().end
    min_val = rand_ranges.min().start

    bin_step = (max_val - min_val) / NB_BUCKETS
    bins = np.array([min_val + i * bin_step for i in range(NB_BUCKETS + 1)])

    histo_appart, mean_nb_overlap = build_belonging_hist(rand_ranges, bins)
    histo_mean_norm = np.copy(histo_appart) / mean_nb_overlap
    histo_local_norm = build_local_norm(rand_ranges, bins)
    histo_low_bound, histo_up_bound = build_bound_hists(rand_ranges, bins)

    return histo_local_norm, histo_mean_norm, histo_low_bound, histo_up_bound, histo_appart, bins

def bound_idx(bins, r):
    idx = 0
    while idx < len(bins) - 1 and bins[idx + 1] < r.start:
        # Haven't reach the first bin of this range
        idx += 1
    start_idx = idx

    while idx +1 < len(bins) - 1 and bins[idx + 1] < r.end:
        # Haven't reach the last bin of this range
        idx += 1
    end_idx = idx
    return end_idx, start_idx

histo_local_norm, histo_mean_norm, histo_low, histo_up, hist_belonging, bins = build_histos(rand_ranges)

In [22]:
print("bins       :", bins[:6], "...", bins[-6:])
print("upper bnds :", histo_up[:6], "...", histo_up[-6:])

bins       : [  0. 100. 200. 300. 400. 500.] ... [ 9500.  9600.  9700.  9800.  9900. 10000.]
upper bnds : [ 2.  3.  2.  6.  6. 11.] ... [191. 186. 198. 205. 196. 188.]


In [23]:
def analyze_counting(hist_belonging, hist_low_bounds, const_ranges, bins):
    nb_line = np.zeros(len(const_ranges))

    for idx, r in enumerate(const_ranges):
        endidx, startidx = bound_idx(bins, r)
        nb_line[idx] += hist_belonging[startidx]  # starting bucket
        for i in range(startidx + 1, endidx + 1): # the rest
            if i >= len(nb_line):
                break
            nb_line[idx] += hist_low_bounds[i]
    return nb_line

def analyze_distri(hist, const_ranges, bins):
    nb_line = np.zeros(len(const_ranges))

    for idx, r in enumerate(const_ranges):
        end_idx, start_idx = bound_idx(bins, r)
        for i in range(start_idx, end_idx + 1):
            if i >= len(hist):
                break
            nb_line[idx] += hist[i]
    return nb_line

real_line_nb = real_overlap(rand_ranges, const_ranges) / NB_RANGES

res_mean_norm = analyze_distri(histo_mean_norm, const_ranges, bins) / NB_RANGES
res_loc_norm = analyze_distri(histo_local_norm, const_ranges, bins) / NB_RANGES
res_counting = analyze_counting(hist_belonging, histo_low, const_ranges, bins) / NB_RANGES

## Results visualization

In [24]:
cols = ["Ref range idx", "Real fraction", "Selectivity", "Delta with real", "Type of approx"]
types_estimation = ["Mean normalization", "Local normalization", "Counting method", "Real fraction"]
datafr = pd.DataFrame(columns=cols)

datafr = add_to_dataframe(datafr, real_line_nb, res_mean_norm, types_estimation[0])
datafr = add_to_dataframe(datafr, real_line_nb, res_loc_norm, types_estimation[1])
datafr = add_to_dataframe(datafr, real_line_nb, res_counting, types_estimation[2])
datafr = add_to_dataframe(datafr, real_line_nb, real_line_nb, types_estimation[3])

In [25]:
fig = px.scatter(datafr, y=cols[3], x=cols[1], color=cols[4],
                 title="Difference between the estimated and the real selectivity as function of the real fraction")

fig.show()